<a href="https://colab.research.google.com/github/abhishek-parashar/corona-virus-analysis/blob/master/corona_virus_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Declaration

### For past few weeks, I have gone through a lot of literature, blogs, codes, videos and what not. 
### The information here is right to my knowledge, this is just a simple analysis and my views for the topic, as a machine learning programmer. 
### Please don't take this blog or code as the sole work on the topic.
### For more information please refer the Government of India site, Research papers, Youtube videos, Github etc. 
### Information provided at the Government of India site https://www.mohfw.gov.in/ is correct and to be referred for the analysis 

I have shared all the links to the dataset, resources for learning about COVID-19, time-series analysis and various lectures on visualization and real-time analysis on my blog post. 
Kindly refer that for further information.
These are the resources where I learned this analysis. 

In [5]:
from zipfile import ZipFile
file_name= "novel-corona-virus-2019-dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [7]:

import json
from datetime import timedelta
from urllib.request import urlopen
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
!pip install geopandas

     |████████████████████████████████| 931kB 3.4MB/s 
     |████████████████████████████████| 10.4MB 12.7MB/s 
     |████████████████████████████████| 14.7MB 29.1MB/s 


In [0]:
import geopandas as gpd

In [10]:
import keras 

Using TensorFlow backend.


In [0]:
import tensorflow as tf

In [12]:
tf.__version__

'2.2.0-rc2'

In [13]:
! ls ../input/covid19-corona-virus-india-dataset

ls: cannot access '../input/covid19-corona-virus-india-dataset': No such file or directory


In [30]:
from zipfile import ZipFile
file_name= "covid19-corona-virus-india-dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [0]:
df = pd.read_csv('complete.csv', parse_dates=['Date'])

In [0]:
df['Name of State / UT'] = df['Name of State / UT'].str.replace('Union Territory of ', '')

In [33]:
df.head()

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
0,2020-01-30,Kerala,1,0,0,10.8505,76.2711,0,1
1,2020-01-31,Kerala,1,0,0,10.8505,76.2711,0,1
2,2020-02-01,Kerala,2,0,0,10.8505,76.2711,0,2
3,2020-02-02,Kerala,3,0,0,10.8505,76.2711,0,3
4,2020-02-03,Kerala,3,0,0,10.8505,76.2711,0,3


In [25]:
df.columns

Index(['Date', 'Name of State / UT', 'Total Confirmed cases (Indian National)',
       'Total Confirmed cases ( Foreign National )',
       'Cured/Discharged/Migrated', 'Latitude', 'Longitude', 'Death',
       'Total Confirmed cases'],
      dtype='object')

In [0]:
# calculating mortality rate. recovery rate and active cases by the reported cases, death, cured discharged.
df = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]
df.columns = ['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Cured']

In [0]:
for i in ['Confirmed', 'Deaths', 'Cured']:
    df[i] = df[i].astype('int')
    
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Cured']
df['Mortality rate'] = df['Deaths']/df['Confirmed']
df['Recovery rate'] = df['Cured']/df['Confirmed']

In [0]:
df = df[['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Active', 'Deaths', 'Mortality rate', 'Cured', 'Recovery rate']]

In [37]:
df.head()

,Date,State/UT,Latitude,Longitude,Confirmed,Active,Deaths,Mortality rate,Cured,Recovery rate
0,2020-01-30,Kerala,10.8505,76.2711,1,1,0,0.0,0,0.0
1,2020-01-31,Kerala,10.8505,76.2711,1,1,0,0.0,0,0.0
2,2020-02-01,Kerala,10.8505,76.2711,2,2,0,0.0,0,0.0
3,2020-02-02,Kerala,10.8505,76.2711,3,3,0,0.0,0,0.0
4,2020-02-03,Kerala,10.8505,76.2711,3,3,0,0.0,0,0.0


In [0]:
# latest cases 
latest = df[df['Date']==max(df['Date'])]

In [0]:
# days
latest_day = max(df['Date'])
day_before = latest_day - timedelta(days = 1)

In [0]:
# state and total cases 
latest_day_df = df[df['Date']==latest_day].set_index('State/UT')
day_before_df = df[df['Date']==day_before].set_index('State/UT')

In [0]:
temp = pd.merge(left = latest_day_df, right = day_before_df, on='State/UT', suffixes=('_lat', '_bfr'), how='outer')
latest_day_df['New cases'] = temp['Confirmed_lat'] - temp['Confirmed_bfr']
latest = latest_day_df.reset_index()
latest.fillna(1, inplace=True)

In [42]:
print(latest_day, day_before)

2020-04-04 00:00:00 2020-04-03 00:00:00


In [0]:
# sorting the latest values in descending order 
temp = latest[['State/UT', 'Confirmed', 'Active', 'New cases', 'Deaths', 'Mortality rate', 'Cured', 'Recovery rate']]
temp = temp.sort_values('Confirmed', ascending=False).reset_index(drop=True)

In [44]:
# ploting the latest values 
temp.style\
    .background_gradient(cmap="Blues", subset=['Confirmed', 'Active', 'New cases'])\
    .background_gradient(cmap="Greens", subset=['Cured', 'Recovery rate'])\
    .background_gradient(cmap="Reds", subset=['Deaths', 'Mortality rate'])

,State/UT,Confirmed,Active,New cases,Deaths,Mortality rate,Cured,Recovery rate
0,Maharashtra,423,362,88.000000,19,0.044917,42,0.099291
1,Tamil Nadu,411,404,102.000000,1,0.002433,6,0.014599
2,Delhi,386,372,167.000000,6,0.015544,8,0.020725
3,Kerala,295,252,9.000000,2,0.006780,41,0.138983
4,Rajasthan,179,176,46.000000,0,0.000000,3,0.016760
5,Uttar Pradesh,174,153,61.000000,2,0.011494,19,0.109195
6,Andhra Pradesh,161,159,29.000000,1,0.006211,1,0.006211
7,Telengana,158,150,51.000000,7,0.044304,1,0.006329
8,Karnataka,128,113,4.000000,3,0.023438,12,0.093750
9,Madhya Pradesh,104,98,5.000000,6,0.057692,0,0.000000


## plotting the Map because it's cool
![alt text](https://drive.google.com/uc?id=1OG6iz85GiIZipkCrEdQrn5ortwt3LVq4)

In [48]:
# map plot
m = folium.Map(location=[20.5937, 78.9629], tiles='cartodbpositron',
               min_zoom=4, max_zoom=6, zoom_start=4)

for i in range(0, len(latest)):
    if latest.iloc[i]['Confirmed']>0:
        folium.Circle(
            location=[latest.iloc[i]['Latitude'], latest.iloc[i]['Longitude']],
            color='#e84545', 
            fill='#e84545',
            tooltip =   '<li><bold>Name of State / UT : '+str(latest.iloc[i]['State/UT'])+
                        '<li><bold>Confirmed cases  : '+str(latest.iloc[i]['Confirmed']),
            radius=int(latest.iloc[i]['Confirmed'])*300).add_to(m)
m            

In [50]:
# plotting the confirmed cases state wise
conf = latest.sort_values('Confirmed', ascending=False)
state_order = conf['State/UT']

fig = px.bar(conf, 
             x="Confirmed", y="State/UT", color='State/UT', title='Confirmed',
             orientation='h', text='Confirmed', height=len(conf)*35,
             color_discrete_sequence = px.colors.qualitative.Vivid)
fig.show()